<a href="https://colab.research.google.com/github/KelvinLam05/price_tracker/blob/main/price_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal of the project**

In ecommerce, it’s very common for retailers to need to monitor the prices of their competitors. Prices make a big difference to sales and if they’re set too high then customers will go elsewhere, so monitoring them is crucial for ensuring the sales keeping coming in.

In this project, we’ll create a price tracker with Webscraper.io. We’ll scrape an ecommerce store, extract the product prices (Black Friday deals), and store them in a CSV file. 

**Scrape the site**

Webscraper.io is a chrome browser extension built for data extraction from webpages. Using this extension we can create a plan (sitemap) how a website should be traversed and what should be extracted.

`{"_id":"black_friday_deals","startUrl":["https://www.belsimpel.nl/telefoon?lang=en&np=1&shipping_country=GB&offer%5B0%5D=Black+Friday&pagina=[1-8]","https://www.belsimpel.nl/tablet?lang=en&offer%5B0%5D=Black+Friday&pagina=[1-2]","https://www.belsimpel.nl/smartwatches?lang=en&offer%5B0%5D=Black+Friday&pagina=[1-3]","https://www.belsimpel.nl/oordopjes?Lang=en&offer%5B0%5D=Black+Friday&pagina=[1-4]","https://www.belsimpel.nl/koptelefoons?lang=en&offer%5B0%5D=Black+Friday&pagina=1"],"selectors":[{"id":"product_wrappers","parentSelectors":["_root"],"type":"SelectorElement","selector":"section.SegmentItemContainerstyle__StyledSegmentItemContainer-sc-10buj0t-0","multiple":true},{"id":"name","parentSelectors":["product_wrappers"],"type":"SelectorText","selector":"h3","multiple":false,"regex":""},{"id":"original_price","parentSelectors":["product_wrappers"],"type":"SelectorText","selector":"span.dgWanj","multiple":false,"regex":""},{"id":"sale_price","parentSelectors":["product_wrappers"],"type":"SelectorText","selector":"span.gxgwxb","multiple":false,"regex":""},{"id":"reviews","parentSelectors":["product_wrappers"],"type":"SelectorText","selector":"div.ReviewSummarystyle__NumberOfReviews-sc-1jkl55v-3","multiple":false,"regex":""},{"id":"rating","parentSelectors":["product_wrappers"],"type":"SelectorText","selector":"div.ReviewSummarystyle__Label-sc-1jkl55v-5","multiple":false,"regex":""}]}`

**Load the data**

In [48]:
# Importing library
import pandas as pd

In [49]:
# Load dataset
df = pd.read_csv('/content/black_friday_deals.csv')

In [50]:
# Only include the relevant columns
df = df[['name', 'original_price', 'sale_price', 'reviews', 'rating']]

In [51]:
# Examine the data
df.head()

,name,original_price,sale_price,reviews,rating
0,Sony WH-1000XM4 Black,€247.00,€227.00,18 verified reviews,Excellent 9.3
1,Beats Solo3 Wireless Black,€209.95,€187.95,10 verified reviews,Excellent 9.1
2,Beats Studio3 Wireless White,€273.95,€261.95,5 verified reviews,Great 8.6
3,Sony WH-1000XM4 Silver,NaN,€266.00,3 verified reviews,Fantastic 10
4,Beats Studio Buds Blue,€189.95,€141.95,2 verified reviews,Excellent 9.3


In [52]:
# Overview of all variables, their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            238 non-null    object
 1   original_price  228 non-null    object
 2   sale_price      238 non-null    object
 3   reviews         238 non-null    object
 4   rating          209 non-null    object
dtypes: object(5)
memory usage: 9.4+ KB


**Data preprocessing**

In [53]:
# Remove the ‘,’ and ‘€’
df['original_price'] = df['original_price'].str.replace(',', '').str.replace('€', '')
df['sale_price'] = df['sale_price'].str.replace(',', '').str.replace('€', '')

In [54]:
# Convert columns to numeric
df['original_price'] = pd.to_numeric(df['original_price'])
df['sale_price'] = pd.to_numeric(df['sale_price'])

In [55]:
# Calculate percent off
df['percent_off'] = round(100 - ((df['sale_price'] / df['original_price']) * 100), 1)

In [56]:
# Remove non-numeric characters from columns 
df['reviews'] = df['reviews'].str.extract('(\d+)')
df['rating'] = df['rating'].str.extract('([-+]?\d*\.?\d+)')

In [57]:
# Convert rating to numeric
df['rating'] = pd.to_numeric(df['rating']) 

In [58]:
# Reorder the columns
df = df[['name', 'original_price', 'sale_price', 'percent_off', 'reviews', 'rating']]

In [59]:
df.head()

,name,original_price,sale_price,percent_off,reviews,rating
0,Sony WH-1000XM4 Black,247.00,227.00,8.1,18,9.3
1,Beats Solo3 Wireless Black,209.95,187.95,10.5,10,9.1
2,Beats Studio3 Wireless White,273.95,261.95,4.4,5,8.6
3,Sony WH-1000XM4 Silver,NaN,266.00,NaN,3,10.0
4,Beats Studio Buds Blue,189.95,141.95,25.3,2,9.3


In [60]:
# Export Pandas DataFrame to CSV
df.to_csv('/content/price_tracker.csv', index = None, header = True)